##Data Loading/Preprocessing 


In [29]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
#tf.enable_eager_execution()
import tempfile
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive #for loading gdrive data
from google.colab import files

import imageio



######KERAS TRY
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


num_classes=2 #for images of numbers, options from 0-9, so 10 classes

batch_size=300  #usually, higher batches give better results

epochs=5 #number of times to go over the full data set

img_rows,img_cols=128,128 #28,28  #size of input images to go into the network

input_shape=(img_rows,img_cols,1)  #1 is the number of channels, 2D image with 
                                   #one channel

#
# Load data from google drive
#

drive.mount('/content/gdrive')

#uploaded = files.upload()




#
# This section might be easier to use, but for now doesn't open correctly
#
fname = '/content/gdrive/My Drive/RCC_DATA/RENAL_CANCER_PROJECT_INPUTS/ONCO_IMAGE_DATA_128x128/10024-0054_2006-12-22/rgb0074.png'

#with open('/content/gdrive/My Drive/RCC_DATA/RENAL_CANCER_PROJECT_INPUTS/RCC_IMAGE_DATA_128x128/10024-0011_2013_05_19/rgb0013.png', 'r') as f:
with open('/content/gdrive/My Drive/RCC_DATA/RENAL_CANCER_PROJECT_INPUTS/ONCO_IMAGE_DATA_128x128/10024-0054_2006-12-22/rgb0074.png', 'r') as f:
  im = imageio.imread(fname)
  print(im.shape)
  #f.write('Hello Google Drive!')
#!cat /content/gdrive/My\ Drive/RCC_DATA/RENAL_CANCER_PROJECT_INPUTS/RCC_IMAGE_DATA_128x128/10024-0511_2013_06_21/rgb0075.png
#!cat /content/gdrive/My\ Drive/foo.txt
  
  
  
  
  

####################
#Keras setup
# dimensions of our images.
img_width, img_height = 128,128

train_data_dir = '/content/gdrive/My Drive/data/train'
validation_data_dir = '/content/gdrive/My Drive/data/validation'
nb_train_samples = 100
nb_validation_samples = 50
epochs = 5 #50
batch_size = 30
#############################################################


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3) #typical RGB setup
    
print('input_shape type is ',type(input_shape))
print('length of input_shape is ',len(input_shape))

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])







##########################

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)


#
# Pull data from gdrive areas
#

print('-----train generation ------\n')
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print('------validation generation ---\n')
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



print('fit generator')
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')


model.summary()


##############################

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

#compile requires 3 main arguments, adam optimizer is the standard
print('metrics names: \n',model.metrics_names)

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)


#model.fit(x_train, y_train,
#          batch_size=batch_size,
#          epochs=epochs,
#          verbose=1,
#          validation_data=(x_test, y_test))

#score = model.evaluate(x_test, y_test, verbose=0)

#from medium.com examples
STEP_SIZE_TEST=train_generator.n//train_generator.batch_size
train_generator.reset()
pred=model.predict_generator(train_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
for xx in predictions:
  if (xx == 'rcc'):
    print('rcc found\n')
###################################################

#score = model.evaluate(train_generator, validation_generator, verbose=0)

#print('Test loss:', score[0])
#print('Test accuracy:', score[1])



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(128, 128, 3)
input_shape type is  <class 'tuple'>
length of input_shape is  3
-----train generation ------

Found 436 images belonging to 2 classes.
------validation generation ---

Found 271 images belonging to 1 classes.
fit generator
Epoch 1/5
3/3 [==============================] - 3s 1s/step - loss: 0.8194 - acc: 0.5333 - val_loss: 0.3451 - val_acc: 1.0000
Epoch 2/5
3/3 [==============================] - 0s 24ms/step - loss: 0.6940 - acc: 0.6556 - val_loss: 0.5888 - val_acc: 1.0000
Epoch 3/5
3/3 [==============================] - 0s 26ms/step - loss: 0.6797 - acc: 0.5667 - val_loss: 0.5715 - val_acc: 1.0000
Epoch 4/5
3/3 [==============================] - 0s 26ms/step - loss: 0.6809 - acc: 0.5778 - val_loss: 0.3790 - val_acc: 1.0000
Epoch 5/5
3/3 [==============================] - 0s 22ms/step - loss: 0.6307 - acc: 0.7078 - val_loss: 0.5166 - val_acc: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:181: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:181: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=3, validation_steps=50)`


Epoch 1/5
3/3 [==============================] - 6s 2s/step - loss: 4.6359e-08 - acc: 0.6333 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/5
3/3 [==============================] - 3s 1s/step - loss: 4.5035e-08 - acc: 0.6778 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/5
3/3 [==============================] - 3s 1s/step - loss: 4.2386e-08 - acc: 0.6667 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/5
3/3 [==============================] - 3s 969ms/step - loss: 4.5035e-08 - acc: 0.6333 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/5
14/14 [==============================] - 2s 163ms/step


In [28]:
a = pred > 0.5
for counter,xx in enumerate(a):
  if(xx =='True'):
      print('found one @',counter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#built-in function convert labels to one-hut encoded, only want binary values
#example, 5 =0 0 0 0 0 1 0 0 0     for digits 0 - 9
#           MSB              LSB
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape)
#print(y_train[0,0,0,0,1])

(60000, 10)


##Create the CNN architecture





In [0]:
l = tf.keras.layers

#32 filters, k = 5
#input image is 28x28x1
model = tf.keras.Sequential([
    l.Conv2D(32, 5, padding='same', activation='relu', input_shape=input_shape),
    #28x28x32 after Conv2D
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.BatchNormalization(),
    l.Conv2D(64,5,padding='same',activation='relu',input_shape=input_shape),
    #14x14x64 going into maxpool,7x7x64 out    
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),   
    #7x7x64 out of maxpool, now flatten into an array
    l.Flatten(),
    #each element is now fully connected to the dense neurons
    l.Dense(1024,activation='relu'), #number of neurons as input
    l.Dropout(0.4), #drop 40%, usually 0.2 to 0.4
    l.Dense(10,activation='softmax'), #
])

#for maxpooling2d, care about kernel size of 2 and stride of 2. This shrinks 
#the size of the image by 2. No filters are changed. There are NO learnable
#parameters in a maxpool operation

#Batchnorm stablizes the network, usually required for most models
#want to flatten the 7x7x64 into a vector of size (7x7x64) elements

#output of flatten goes into hidden neurons, which are fully connected. These
#fully connected layers are connected to each element of the input. Each neuron
# is w(x) + b. Number of neurons in layer depends on task and testing
#next layer is softmax layer, which has the same neurons as the number of labels
#expected. Each output node is the probability that the image info is of that
#label type

#dropout is only used during training, not testing


model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)    

##Compile and Train the model

In [0]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

#compile requires 3 main arguments, adam optimizer is the standard

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0258 - acc: 0.9915 - val_loss: 0.0308 - val_acc: 0.9910
Epoch 2/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0223 - acc: 0.9931 - val_loss: 0.0260 - val_acc: 0.9928
Epoch 3/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0177 - acc: 0.9944 - val_loss: 0.0309 - val_acc: 0.9900
Epoch 4/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0139 - acc: 0.9956 - val_loss: 0.0305 - val_acc: 0.9912
Epoch 5/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0107 - acc: 0.9967 - val_loss: 0.0299 - val_acc: 0.9923
Test loss: 0.02993167931542116
Test accuracy: 0.9923
